# Import of packages

In [1]:
'''To import the required packages.'''
import pandas as pd
import numpy as np
import networkx as nx
import collections
import matplotlib.pyplot as plt
import math
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from kneed import KneeLocator
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA

#import osmnx as ox

# Settings

In [2]:
'''To display all output results of a Jupyter cell.'''
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
'''To ensure that the output results of extensive output results are not truncated.'''
#pd.options.display.max_rows = 4000

'To ensure that the output results of extensive output results are not truncated.'

In [4]:
'''To change the width of the Notebook to see the output on the screen'''
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

'To change the width of the Notebook to see the output on the screen'

# **Swiss railway system**

# Import of the Swiss railway datasets

In [5]:
'''To register the GitHub link with the Swiss data as a variable.'''
datalink = "https://raw.githubusercontent.com/polkuleuven/Thesis_Train/main/gtfs_train_Switzerland_1503/"

'To register the GitHub link with the Swiss data as a variable.'

In [6]:
'''Import all the GTFS data'''
#To import the agency dataset that contains limited information about the Swiss SBB railway agency.
agency_Switzerland = pd.read_csv(datalink + "agency.txt", sep=",")
#To import the stops dataset that contains information about the ids, the names and the geographical coordinates of the Swiss SBB railway stations.
stops_Switzerland = pd.read_csv(datalink + "stops.txt", sep=",")
#To import the feed_info dataset that contains limited information about the Swiss SBB railway feed.
feed_info_Switzerland = pd.read_csv(datalink + "feed_info.txt", sep=",")
#To import the transfers dataset that gives the minimum transfer time to switch routes at each Swiss SBB railway station.
transfers_not_cleaned_Switzerland = pd.read_csv(datalink + "transfers.txt", sep=",")
#To import the routes dataset that provides the id, the name and the type of vehicle used for all Swiss SBB railway routes.
routes_Switzerland = pd.read_csv(datalink + "routes.txt", sep=",")
#To import the trips dataset that gives for all routes an overview of the trips and the headsigns of these trips belonging to the Swiss SBB railway route.
#The service_id is an indication of all the dates this trip is valid (consultable in the calendar_dates dataset).
trips_Switzerland = pd.read_csv(datalink + "trips.txt", sep=",")
#To import the stop_times dataset that gives for all trips an overview of the ids of the stations served and the sequence in which these stations are served. 
#In addition, for all the trips the arrival and departure times at the stations served are given.
stop_times_Switzerland = pd.read_csv(datalink + "stop_times.txt", sep=",")
#To import the calendar dataset that gives the first and last date of all data observations.
calendar_Switzerland = pd.read_csv(datalink + "calendar.txt", sep=",")
#To import the calendar_dates dataset that gives for each service_id all the exact dates when that service_id is valid.
calendar_dates_Switzerland = pd.read_csv(datalink + "calendar_dates.txt", sep=",")

'Import all the GTFS data'

/Users/pol/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Cleaning of the Swiss railway data

In [7]:
'''To clean the routes_Switzerland df'''
#To keep the train routes
routes_cleaned_Switzerland = routes_Switzerland[routes_Switzerland['route_type'] == 2]
routes_cleaned_Switzerland

'To clean the routes_Switzerland df'

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,04236.06____.020:4236,06____,RE,RE 4236,NaN,2,NaN,NaN,NaN
1,04700.06____.001:4700,06____,RE,RE 4700,NaN,2,NaN,NaN,NaN
2,04700.06____.014:4700,06____,RE,RE 4700,NaN,2,NaN,NaN,NaN
3,04701.06____.002:4701,06____,RE,RE 4701,NaN,2,NaN,NaN,NaN
4,04701.06____.015:4701,06____,RE,RE 4701,NaN,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
49346,87945.L7____.001:5,L7____,S5,S 5,NaN,2,NaN,FFEA00,000000
49347,87946.L7____.001:5,L7____,S5,S 5,NaN,2,NaN,FFEA00,000000
49348,87947.L7____.001:5,L7____,S5,S 5,NaN,2,NaN,FFEA00,000000
49349,87948.L7____.001:5,L7____,S5,S 5,NaN,2,NaN,FFEA00,000000


In [8]:
'''To clean the calendar_dates_Switzerland df'''
#To filter the dates from the selected begin to the end date
begin_date = 20210314
end_date = 20210713
calendar_dates_cleaned_Switzerland = calendar_dates_Switzerland.copy()
calendar_dates_cleaned_Switzerland = calendar_dates_cleaned_Switzerland.drop(calendar_dates_cleaned_Switzerland[(calendar_dates_cleaned_Switzerland['date'] > end_date) |(calendar_dates_cleaned_Switzerland['date'] < begin_date)].index)
calendar_dates_cleaned_Switzerland

'To clean the calendar_dates_Switzerland df'

,service_id,date,exception_type
63,5,20210315,1
64,5,20210316,1
65,5,20210317,1
66,5,20210318,1
67,5,20210319,1
...,...,...,...
1536188,12559,20210711,1
1536189,12559,20210712,1
1536190,12559,20210713,1
1536341,12560,20210603,1


In [9]:
'''To clean the stop_times_Switzerland df'''
# To remove the superfluous characters of the stop_id (platform codes)
stop_times_cleaned_Switzerland = stop_times_Switzerland.copy()
stop_times_cleaned_Switzerland_column = stop_times_cleaned_Switzerland['stop_id'].str.split(':').str[0]
stop_times_cleaned_Switzerland.loc[:,'stop_id'] = stop_times_cleaned_Switzerland_column

# To make the stop_ids numerical 
stop_times_cleaned_Switzerland.loc[:,'stop_id'] = stop_times_cleaned_Switzerland.loc[:,'stop_id'].astype(np.int64)
stop_times_cleaned_Switzerland

'To clean the stop_times_Switzerland df'

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,attributes_ch
0,0:1,18:16:00,18:16:00,8014554,0,NaN,0,0,NaN,NaN
1,0:1,18:28:00,18:30:00,8014534,1,NaN,0,0,NaN,NaN
2,0:1,18:40:00,18:40:00,8014529,2,NaN,0,0,NaN,NaN
3,0:1,18:49:00,18:50:00,8014521,3,NaN,0,0,NaN,NaN
4,0:1,18:58:00,18:59:00,8014518,4,NaN,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1034556,49350:2,10:38:00,10:38:00,8069220,3,NaN,3,3,NaN,X
1034557,49350:2,10:40:00,10:40:00,8014429,4,NaN,3,3,NaN,X
1034558,49350:2,10:41:00,10:42:00,8060979,5,NaN,3,3,NaN,X
1034559,49350:2,10:43:00,10:43:00,8060978,6,NaN,3,3,NaN,X


In [10]:
'''To clean the stops_Switzerland df '''
#To remove the superfluous characters (platform codes)
stops_cleaned_Switzerland_column = stops_Switzerland['stop_id'].str.split(':').str[0]
stops_cleaned_Switzerland = stops_Switzerland.copy()
stops_cleaned_Switzerland.loc[:,'stop_id'] = stops_cleaned_Switzerland_column

#To make the stop_ids numerical and to remove the duplicate stop_ids
stops_cleaned_Switzerland = stops_cleaned_Switzerland[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
stops_cleaned_Switzerland.loc[:,'stop_id'] = stops_cleaned_Switzerland.loc[:,'stop_id'].astype(np.int64)
stops_cleaned_Switzerland = stops_cleaned_Switzerland.drop_duplicates()

#To remove the accents from the stop_name and to change to uppercase
stops_cleaned_Switzerland.loc[:,'stop_name'] = stops_cleaned_Switzerland.loc[:,'stop_name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
stops_cleaned_Switzerland.loc[:,'stop_name'] = stops_cleaned_Switzerland.loc[:,'stop_name'].str.upper()

#To initialize the Nominatim API to get the location from the input string 
geolocator = Nominatim(user_agent="application")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=0.2)

#To get the location with the geolocator.reverse() function and to extract the country from the location instance
country_list = []
for index, row in stops_cleaned_Switzerland.iterrows():
    latitude = row['stop_lat']
    longitude = row['stop_lon']
    # To assign the latitude and longitude into a geolocator.reverse() method
    location = reverse((latitude, longitude), language='en', exactly_one=True)
    # To get the country from the given list and parsed into a dictionary with raw function()
    address = location.raw['address']
    country = address.get('country', '')
    country_list.append(country)

#To add the values of country_list as a new attribute country     
stops_cleaned_Switzerland.loc[:,'country'] = country_list
stops_cleaned_Switzerland

#To calculate the total number of Belgian stations in the stops_cleaned dataset
swiss_stops_Switzerland = stops_cleaned_Switzerland[stops_cleaned_Switzerland['country'] == 'Switzerland']
swiss_stops_Switzerland_series = stops_cleaned_Switzerland.loc[stops_cleaned_Switzerland['country'] == 'Switzerland', 'stop_name']
#stops_cleaned_Switzerland.to_csv('stops_cleaned_Switzerland.csv')

'To clean the stops_Switzerland df '

RateLimiter caught an error, retrying (0/2 tries). Called with (*((47.881166, 8.397886),), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/pol/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto
    return 

RateLimiter caught an error, retrying (0/2 tries). Called with (*((47.975496, 7.803006),), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/pol/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto
    return 

RateLimiter caught an error, retrying (0/2 tries). Called with (*((47.960359999999994, 7.76818),), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/pol/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto
   

RateLimiter caught an error, retrying (1/2 tries). Called with (*((47.919721, 7.6976320000000005),), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/pol/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto
 

...

RateLimiter caught an error, retrying (1/2 tries). Called with (*((46.337109999999996, 7.038479),), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/pol/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto
  

RateLimiter caught an error, retrying (1/2 tries). Called with (*((46.36124, 7.052789),), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/pol/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto
    return s

RateLimiter caught an error, retrying (0/2 tries). Called with (*((46.35116, 7.155019),), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/pol/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto
    return s

RateLimiter caught an error, retrying (0/2 tries). Called with (*((46.339259999999996, 7.006049000000001),), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/pol/opt/anaconda3/lib/python3.8/socket.py", line 669, in re

KeyboardInterrupt: 

In [ ]:
#stops_cleaned_Switzerland = pd.read_csv("stops_cleaned_Switzerland.csv", sep=",")

### To merge the files

In [11]:
'''To select all required fields'''
agency_cleaned_Switzerland = agency_Switzerland[['agency_id', 'agency_name', 'agency_url', 'agency_timezone']]
routes_cleaned_Switzerland = routes_cleaned_Switzerland[['route_id', 'agency_id', 'route_short_name', 'route_long_name', 'route_type']]
trips_cleaned_Switzerland = trips_Switzerland[['trip_id', 'route_id', 'service_id', 'trip_headsign']]
calendar_dates_cleaned_Switzerland = calendar_dates_cleaned_Switzerland[['service_id', 'date']]
stops_cleaned_Switzerland = stops_cleaned_Switzerland[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
stop_times_cleaned_Switzerland = stop_times_cleaned_Switzerland[['trip_id', 'stop_id', 'arrival_time', 'departure_time', 'stop_sequence']]

'To select all required fields'

In [12]:
''' To merge the Swiss files '''
#To merge the stop_times df with the stops df on stop_id
stop_times_stops_Switzerland = pd.merge(stop_times_cleaned_Switzerland, stops_cleaned_Switzerland[['stop_id','stop_name', 'stop_lat', 'stop_lon']], on='stop_id')

#To merge the trips df with the routes df on route_id
routes_trips_Switzerland = pd.merge(routes_cleaned_Switzerland[['route_id']], trips_cleaned_Switzerland, on='route_id')

#To merge the stop_times_stops df with the trips_routes df on trip_id
routes_trips_stop_times_stops_Switzerland = pd.merge(routes_trips_Switzerland, stop_times_stops_Switzerland, on='trip_id')

#To take only the service_ids present in both the routes_trips_stop_times_stops df and the calendar_dates df into account
calendar_dates_cleaned_unique_Switzerland = calendar_dates_cleaned_Switzerland['service_id'].unique()
routes_trips_stop_times_stops_calendar_dates_Switzerland = routes_trips_stop_times_stops_Switzerland[(routes_trips_stop_times_stops_Switzerland['service_id'].isin(calendar_dates_cleaned_unique_Switzerland))]

' To merge the Swiss files '

# Preparation space-of-stops

In [ ]:
# Verder te gaan met routes_trips_stop_times_stops_calendar_dates_Switzerland of eventueel te hernoemen

In [13]:
routes_trips_stop_times_stops_calendar_dates_Switzerland

,route_id,trip_id,service_id,trip_headsign,stop_id,arrival_time,departure_time,stop_sequence,stop_name,stop_lat,stop_lon
0,04236.06____.020:4236,0:1,19311,Karlsruhe Hbf,8014554,18:16:00,18:16:00,0,ENGEN,47.856347,8.772786
1,04236.06____.020:4236,0:1,19311,Karlsruhe Hbf,8014534,18:28:00,18:30:00,1,IMMENDINGEN,47.936007,8.729536
2,04236.06____.020:4236,0:1,19311,Karlsruhe Hbf,8014529,18:40:00,18:40:00,2,DONAUESCHINGEN,47.947786,8.498919
3,04236.06____.020:4236,0:1,19311,Karlsruhe Hbf,8014521,18:49:00,18:50:00,3,VILLINGEN (SCHWARZW),48.058022,8.465261
4,04236.06____.020:4236,0:1,19311,Karlsruhe Hbf,8014518,18:58:00,18:59:00,4,ST GEORGEN (SCHWARZW),48.123813,8.341955
...,...,...,...,...,...,...,...,...,...,...,...
991431,87949.L7____.001:5,49350:2,56518,Weil am Rhein,8014429,10:40:00,10:40:00,4,WEIL AM RHEIN-OST,47.590629,7.634780
991432,87949.L7____.001:5,49350:2,56518,Weil am Rhein,8069220,10:38:00,10:38:00,3,LORRACH DAMMSTRASSE,47.597713,7.655224
991433,87949.L7____.001:5,49350:2,56518,Weil am Rhein,8014440,10:37:00,10:37:00,2,LORRACH-STETTEN,47.601418,7.659032
991434,87949.L7____.001:5,49350:2,56518,Weil am Rhein,8069221,10:35:00,10:35:00,1,LORRACH MUSEUM/BURGHOF,47.607961,7.661196
